In [ ]:
import sys
sys.path.append('../../preprocess_assets/')

In [ ]:
import glob
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding
import tensorflow as tf
from features_extraction import *
from data_shuffling_split import *
from ara_vec_preprocess_configs import *
from ml_modeling import *
from keras_models import *

In [ ]:
strat_train_set = read_file("train/strat_train_set.csv")
strat_train_set.head()

In [4]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

The number of instances in the training data after StratifiedShuffleSplit are:  9608
The number of instances in the testing data after StratifiedShuffleSplit are:   197
The number of trainin instances:  9608
The number of validation instances:  197
The number of trainin labels :  9608
The number of validation labels :  197


In [5]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

fullgram_x_train_text_tokenized = get_all_ngrams(x_train_text_tokenized)
print("full gram tokenization : \n", fullgram_x_train_text_tokenized[:3])
print("="*50)
fullgram_x_val_text_tokenized = get_all_ngrams(x_val_text_tokenized)
print("full gram tokenization : \n", fullgram_x_val_text_tokenized[:3])
print("="*50)

Before Tokenization : 
 [' 3 : 00 عصرا مع يوسف الحسيني تابعونا …', 'بمناسبه #الامتحانات اللي اتاجلت و #رجعوا في كلامهم تاني -_- #حقوق_انجليزي_لحد_ما_يجيبوا_مدرس_انجليزي', '#باسم_طلع_حرامي باسييم مهزوز علشان اتفيش']
After Tokenization : 
 [['3', ':', '00', 'عصرا', 'مع', 'يوسف', 'الحسيني', 'تابعونا', '…'], ['بمناسبه', '#', 'الامتحانات', 'اللي', 'اتاجلت', 'و', '#', 'رجعوا', 'في', 'كلامهم', 'تاني', '-_-', '#', 'حقوق_انجليزي_لحد_ما_يجيبوا_مدرس_انجليزي'], ['#', 'باسم_طلع_حرامي', 'باسييم', 'مهزوز', 'علشان', 'اتفيش']]
Before Tokenization : 
 ['_ 25 جميع ارقام #الاهلي #مصر في #دوري_ابطال_افريقيا من البدايه وحتي 2017 ', '# الانتخابات العراقيه في مرمي نيران داعش : بغداد - ارتفع منسوب العنف في العراق مع اقتراب الموعد #دبي', 'الطوابير اليوم عند المقار الانتخابيه اقصر من اليوم الاول في المرحله الاولي شاركوا ارجوكم، مشاركتكم نجحت الاولي وهتنجح الثانيه ان شاء الله']
After Tokenization : 
 [['_', '25', 'جميع', 'ارقام', '#', 'الاهلي', '#', 'مصر', 'في', '#', 'دوري_ابطال_افريقيا', 'من', 'البدايه', 'وحتي',

# Our CBOW Word2Vec Model

In [6]:
our_word2vec_model = load_word2vec_model("../word2vec_models/rezk/skipgram/skip_gram_fullgram_vec_size_300-d_min_count_100")

In [7]:
number_of_features = 300
max_len_str = 132
word2vec_path = "rezk/cbow/"
model_path_to_save = "../ml_models_saved/"
hid_num_neurons = 25
learning_rate = .00005
epochs = 10
estimators = voting_models()

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
RMSprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


X_train_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)
# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(9608, 132, 300)
(9608, 39600)
[-0.03217  -0.01317  -0.2024   -0.06274   0.2361    0.0524   -0.0534
  0.02296   0.1979    0.03778  -0.07263   0.0847    0.1415    0.04736
 -0.03717   0.03058   0.2634    0.1676    0.2079   -0.1761    0.11365
 -0.11273  -0.00891   0.0977    0.0878    0.007427  0.05176  -0.287
 -0.1898    0.1548    0.1254    0.1622   -0.00457  -0.03232  -0.0723
 -0.3003    0.02701  -0.2847    0.0875   -0.008804 -0.09656  -0.04337
  0.13      0.03595   0.1059    0.3813   -0.1231   -0.02145   0.0797
 -0.03735 ]
(197, 132, 300)
(197, 39600)
[-0.0417   -0.1654   -0.2788   -0.1664    0.3179   -0.02107  -0.1172
  0.0637    0.2058    0.161     0.00487  -0.04663   0.02643   0.1666
  0.03102  -0.02977   0.4348    0.188    -0.03784   0.006496  0.01959
 -0.02646   0.0972    0.0683   -0.2998   -0.07324  -0.0563   -0.377
 -0.06244   0.0668    0.1251    0.2404   -0.189    -0.1329    0.3687
  0.2095    0.3115    0.04218   0.003723  0.1471   -0.3901   -0.1722
  0.05817  -0.1687   -0.1571 

# Our CBOW Word2Vec Model

# With  Rmsprob and  Batch Normalization

In [8]:
callbacks_ = keras_callbacks(word2vec_type="rezk_skipgram_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

rezk_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_
../test_models/ml_models_saved/dl_models/tensor_logs/run_2022_05_10_05_53_54_rezk_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_


2022-05-10 05:53:54.605220: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 05:53:54.615256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 05:53:54.617094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 05:53:55.059442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 05:53:55.061236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 132, 300)         1200      
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 132, 25)           32600     
                                                                 
 batch_normalization_1 (Batc  (None, 132, 25)          100       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 132, 25)           0         
                                                                 
 flatten (Flatten)           (None, 3300)              0         
                                                                 
 dense (Dense)               (None, 18)                5

In [9]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/10


2022-05-10 05:54:07.003769: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


301/301 [==============================] - 13s 27ms/step - loss: 1.4904 - accuracy: 0.5541 - val_loss: 1.4810 - val_accuracy: 0.5685 - lr: 5.0000e-05
Epoch 2/10
301/301 [==============================] - 7s 24ms/step - loss: 1.1122 - accuracy: 0.6170 - val_loss: 1.0856 - val_accuracy: 0.6193 - lr: 5.0000e-05
Epoch 3/10
301/301 [==============================] - 7s 23ms/step - loss: 1.0373 - accuracy: 0.6394 - val_loss: 1.0453 - val_accuracy: 0.6396 - lr: 5.0000e-05
Epoch 4/10
301/301 [==============================] - 7s 23ms/step - loss: 0.9858 - accuracy: 0.6509 - val_loss: 1.0289 - val_accuracy: 0.6701 - lr: 5.0000e-05
Epoch 5/10
301/301 [==============================] - 7s 23ms/step - loss: 0.9452 - accuracy: 0.6584 - val_loss: 0.9981 - val_accuracy: 0.6751 - lr: 5.0000e-05
Epoch 6/10
301/301 [==============================] - 7s 23ms/step - loss: 0.9142 - accuracy: 0.6682 - val_loss: 0.9971 - val_accuracy: 0.6345 - lr: 5.0000e-05
Epoch 7/10
301/301 [==============================

# Bakr CBOW Word2Vec Model

In [10]:
bakr_word2vec_model = load_word2vec_model("../word2vec_models/bakr/skipgram/full_grams_sg_300_twitter.mdl")

In [11]:
number_of_features = 300
max_len_str = 132
word2vec_path = "bakr/cbow/"
model_path_to_save = "../ml_models_saved/"
hid_num_neurons = 25
learning_rate = .00005
epochs = 10
estimators = voting_models()

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
RMSprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


X_train_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)
# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(9608, 132, 300)
(9608, 39600)
[ 0.0719   -0.3208   -0.296     0.3457   -0.09485  -0.2087    0.2418
  0.4934    0.4187    0.1384   -0.1022    0.1649    0.05237   0.41
  0.4453   -0.4553    0.5864   -0.1599    0.1877   -0.1472    0.202
  0.1216   -0.12006  -0.1691   -0.2438    0.2272    0.0591   -0.468
  0.05576  -0.3276   -0.5435    0.2308   -0.3484    0.587    -0.2744
  0.4648    0.164    -0.2766    0.002684  0.3855   -0.1882    0.3435
  0.1125    0.2607   -0.3838   -0.0414    0.3545    0.4075    0.0791
 -0.1009  ]
(197, 132, 300)
(197, 39600)
[ 0.194   -0.04285  0.1187   0.6753  -0.5776   0.03906  0.3904  -0.06036
  0.0907  -0.1181  -0.1202   0.05658  0.0953  -0.0804  -0.077    0.11505
 -0.7163   0.1592  -0.282    0.5913  -0.3784  -0.1824  -0.3767  -0.2401
 -0.2527  -0.3943  -0.0648   0.1337   0.1364   0.04095 -0.05072  0.3215
 -0.07935  0.0795  -0.11865  0.08307 -0.03586 -0.387    0.3235  -0.39
  0.3655   0.04654  0.5005   0.4187  -0.2798   0.01662  0.01596  0.1388
 -0.3381   0.4678

# With  Rmsprob and  Batch Normalization

In [12]:
callbacks_ = keras_callbacks(word2vec_type="bakr_skipgram_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

bakr_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_
../test_models/ml_models_saved/dl_models/tensor_logs/run_2022_05_10_05_56_46_bakr_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 132, 300)         1200      
 hNormalization)                                                 
                                                                 
 lstm_1 (LSTM)               (None, 132, 25)           32600     
                                                                 
 batch_normalization_3 (Batc  (None, 132, 25)          100       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 132, 25)           0         
                                       

In [13]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/10
301/301 [==============================] - 11s 28ms/step - loss: 1.4156 - accuracy: 0.5657 - val_loss: 1.9369 - val_accuracy: 0.4061 - lr: 5.0000e-05
Epoch 2/10
301/301 [==============================] - 8s 27ms/step - loss: 1.1535 - accuracy: 0.6166 - val_loss: 1.1017 - val_accuracy: 0.6294 - lr: 5.0000e-05
Epoch 3/10
301/301 [==============================] - 8s 25ms/step - loss: 1.0818 - accuracy: 0.6305 - val_loss: 1.0789 - val_accuracy: 0.6650 - lr: 5.0000e-05
Epoch 4/10
301/301 [==============================] - 8s 25ms/step - loss: 1.0216 - accuracy: 0.6400 - val_loss: 1.0435 - val_accuracy: 0.6447 - lr: 5.0000e-05
Epoch 5/10
301/301 [==============================] - 7s 25ms/step - loss: 0.9804 - accuracy: 0.6519 - val_loss: 1.0305 - val_accuracy: 0.6802 - lr: 5.0000e-05
Epoch 6/10
301/301 [==============================] - 7s 25ms/step - loss: 0.9462 - accuracy: 0.6584 - val_loss: 1.0070 - val_accuracy: 0.6548 - lr: 5.0000e-05
Epoch 7/10
301/301 [===================

# Muhammed CBOW Word2Vec Model

In [14]:
muhammed_word2vec_model = load_word2vec_model("../word2vec_models/muhammed/skipgram/w2v_SG_300_3_400_10.model")

In [15]:
number_of_features = 300
max_len_str = 132
word2vec_path = "muhammed/cbow/"
model_path_to_save = "../ml_models_saved/"
hid_num_neurons = 25
learning_rate = .00005
epochs = 10
estimators = voting_models()

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
SGD_optimizer     =tf.keras.optimizers.SGD(learning_rate=learning_rate)
Adam_optimizer = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999)
RMSprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


X_train_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)
# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(9608, 132, 300)
(9608, 39600)
[-0.1059    0.779     0.5645    0.2186    0.064    -0.383     0.03687
  0.3027   -0.02603  -0.1044   -0.1112   -0.1466   -0.1012   -0.1087
 -0.03818  -0.2778   -0.0231   -0.3596    0.0175    0.437    -0.1844
  0.3684    0.2272   -0.1696    0.0724    0.1407    0.2087    0.5664
  0.3533   -0.189    -0.1112    0.3716   -0.8423    0.2876   -0.2793
 -0.412    -0.1206    0.3132    0.626    -0.086    -0.1621   -0.1791
  0.003435  0.1247   -0.344     0.5044    0.513     0.1846   -0.1996
  0.1203  ]
(197, 132, 300)
(197, 39600)
[-0.1069     0.1539     0.0705     0.11505   -0.1282     0.1267
  0.1873     0.0582    -0.2014     0.275     -0.1477    -0.1154
 -0.01935   -0.02487    0.4915     0.368      0.4028    -0.0008845
 -0.09784    0.426      0.01115    0.1553     0.01869   -0.01886
  0.08734   -0.3423     0.233      0.266      0.2019     0.3665
  0.06775    0.1458     0.04166    0.1588     0.1333     0.1191
 -0.07104    0.2426     0.2003    -0.12445    0.2161    

# With  Rmsprob and  Batch Normalization

In [16]:
callbacks_ = keras_callbacks(word2vec_type="muhammed_skipgram_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

muhammed_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_
../test_models/ml_models_saved/dl_models/tensor_logs/run_2022_05_10_05_58_16_muhammed_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Batc  (None, 132, 300)         1200      
 hNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 132, 25)           32600     
                                                                 
 batch_normalization_5 (Batc  (None, 132, 25)          100       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 132, 25)           0         
                               

In [17]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/10
301/301 [==============================] - 11s 26ms/step - loss: 1.4786 - accuracy: 0.5568 - val_loss: 4.7477 - val_accuracy: 0.0000e+00 - lr: 5.0000e-05
Epoch 2/10
301/301 [==============================] - 7s 23ms/step - loss: 1.1690 - accuracy: 0.6123 - val_loss: 1.1037 - val_accuracy: 0.6142 - lr: 5.0000e-05
Epoch 3/10
301/301 [==============================] - 7s 23ms/step - loss: 1.0933 - accuracy: 0.6326 - val_loss: 1.0886 - val_accuracy: 0.6447 - lr: 5.0000e-05
Epoch 4/10
301/301 [==============================] - 7s 23ms/step - loss: 1.0321 - accuracy: 0.6431 - val_loss: 1.0488 - val_accuracy: 0.6599 - lr: 5.0000e-05
Epoch 5/10
301/301 [==============================] - 7s 23ms/step - loss: 0.9981 - accuracy: 0.6459 - val_loss: 1.0445 - val_accuracy: 0.6548 - lr: 5.0000e-05
Epoch 6/10
301/301 [==============================] - 7s 23ms/step - loss: 0.9707 - accuracy: 0.6551 - val_loss: 1.0186 - val_accuracy: 0.6599 - lr: 5.0000e-05
Epoch 7/10
301/301 [===============

# Load best model & predict test set

In [18]:
strat_test_set = read_file("test/strat_test_set.csv")
strat_test_set.head()

,text,label,classes
0,نرجوا عدم متابعه وحظر حسابات : المباحث تابعني ...,2,OBJ
1,المسلماني اغلي متحدث اعلامي للرئيس واشهر كذاب ...,0,NEG
2,الاحد 3 يوليو : ابو الفتوح في ندوه بالمؤتمر ال...,2,OBJ
3,#كبسوله_صحيه #صحه #طفل #طفلي #نوم ##نوبه #نصيحه,2,OBJ
4,#اللي_رافضين_السيسي_رئيس_بيعملوا_فولوا_لبعض #م...,1,NEU


In [19]:
X_test_text = list(strat_test_set['text'])
y_test = strat_test_set['label'].values

In [20]:
X_test_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(X_test_text)

print("Before Tokenization : \n", X_test_text[:3])
print("="*50)
print("After Tokenization : \n", X_test_text_tokenized[:3])
print("="*50)

fullgram_X_test_text_tokenized = get_all_ngrams(X_test_text_tokenized)
print("full gram tokenization : \n", fullgram_X_test_text_tokenized[:3])
print("="*50)

Before Tokenization : 
 ['نرجوا عدم متابعه وحظر حسابات : المباحث تابعني واتابعك زياده المتابعين الاخبار المحليه #الشعب_يقول_كلمته #جماعه_انصار_بيت_طنيطر #الرياض #جده', 'المسلماني اغلي متحدث اعلامي للرئيس واشهر كذاب ومنافق ومضلل ومحرض تحول الي قليل الادب وضع معارضي الجيش في سله القمامه كلام زباله', 'الاحد 3 يوليو : ابو الفتوح في ندوه بالمؤتمر السنوي لهندسه عين شمس 5 مساء']
After Tokenization : 
 [['نرجوا', 'عدم', 'متابعه', 'وحظر', 'حسابات', ':', 'المباحث', 'تابعني', 'واتابعك', 'زياده', 'المتابعين', 'الاخبار', 'المحليه', '#', 'الشعب_يقول_كلمته', '#', 'جماعه_انصار_بيت_طنيطر', '#', 'الرياض', '#', 'جده'], ['المسلماني', 'اغلي', 'متحدث', 'اعلامي', 'للرئيس', 'واشهر', 'كذاب', 'ومنافق', 'ومضلل', 'ومحرض', 'تحول', 'الي', 'قليل', 'الادب', 'وضع', 'معارضي', 'الجيش', 'في', 'سله', 'القمامه', 'كلام', 'زباله'], ['الاحد', '3', 'يوليو', ':', 'ابو', 'الفتوح', 'في', 'ندوه', 'بالمؤتمر', 'السنوي', 'لهندسه', 'عين', 'شمس', '5', 'مساء']]
full gram tokenization : 
 [['نرجوا', 'عدم', 'متابعه', 'وحظر', 'حسابات', ':'

In [23]:
rezk_model = keras_load_model("../ml_models_saved/dl_models/run_with_rezk_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05__model.h5"  )
bakr_model = keras_load_model("../ml_models_saved/dl_models/run_with_bakr_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05__model.h5"  )
muhammed_model = keras_load_model("../ml_models_saved/dl_models/run_with_muhammed_skipgram_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05__model.h5"  )

X_test_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
X_test_embed_matrix = X_test_embed_matrix.reshape(X_test_embed_matrix.shape[0], max_len_str, number_of_features)

keras_f1_score_result(rezk_model, X_test_embed_matrix, y_test)
print("="*50)

X_test_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
X_test_embed_matrix = X_test_embed_matrix.reshape(X_test_embed_matrix.shape[0], max_len_str, number_of_features)
keras_f1_score_result(bakr_model, X_test_embed_matrix, y_test)
print("="*50)

X_test_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
X_test_embed_matrix = X_test_embed_matrix.reshape(X_test_embed_matrix.shape[0], max_len_str, number_of_features)
keras_f1_score_result(muhammed_model, X_test_embed_matrix, y_test)

(201, 132, 300)
(201, 39600)
[-0.1384   -0.1466    0.0389    0.2112    0.3079    0.1342    0.1416
  0.2318   -0.1287   -0.1637   -0.0872    0.1997    0.00798   0.09326
 -0.3176   -0.1678    0.388     0.5825   -0.10803   0.232    -0.276
  0.03293   0.178    -0.2404   -0.01758  -0.158     0.02231  -0.2222
 -0.3208   -0.2766   -0.2998   -0.08307  -0.1837    0.013504  0.3416
  0.3245   -0.533     0.003767  0.3345    0.1655   -0.03128   0.035
  0.2615   -0.3096   -0.7563    0.1377    0.143     0.2815   -0.1858
  0.01923 ]
===================== Validate Result =====================
F1 score is:  0.6716417910447762
(201, 132, 300)
(201, 39600)
[-2.0544e-01  1.7297e-01  2.9639e-01  1.8506e-01 -3.7445e-02 -2.1271e-02
  3.2715e-01  1.3477e-01  1.0474e-01 -1.0223e-01  1.3721e-01  6.7139e-04
 -2.5171e-01  4.9829e-01  5.2490e-02 -2.9443e-01 -1.0754e-01 -5.1193e-03
  1.7749e-01 -7.4902e-01  3.2898e-02  2.8052e-01 -5.2307e-02  3.1445e-01
 -1.0071e-01 -1.4087e-01  7.8809e-01 -1.9531e-02  2.8394e-01  7

0.647